In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all' #默认为'last

# 第 10 章 数据聚合与分组运算

* 计算分组摘要统计，如计数、平均值、标准差，或用户自定义函数。 
* 计算分组的概述统计，比如数量、平均值或标准差，或是用户定义的函数。
* 应用组内转换或其他运算，如规格化、线性回归、排名或选取子集等。
* 计算透视表或交叉表。
* 执行分位数分析以及其它统计分组分析。

## 10.1GroupBy机制

In [3]:
df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two','one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-2.063758,-1.030364
1,a,two,-0.704522,1.180135
2,b,one,-2.145779,0.138698
3,b,two,1.645362,-1.050341
4,a,one,0.335414,0.747669


#### 分组键是Series：因为是从DataFrame中提取出来的

##### 分组键是Series，一次传入一个数组的列表

In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped
#grouped是一个GroupBy对象，还没进行任何计算，只是含有一些有关分组键df['key']的中间数据而已。

In [5]:
grouped.mean()

key1
a   -0.810956
b   -0.250209
Name: data1, dtype: float64

##### 分组键为Series：一次传入多个数组的列表

In [10]:

means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means
means.unstack()

key1  key2
a     one    -0.864172
      two    -0.704522
b     one    -2.145779
      two     1.645362
Name: data1, dtype: float64

key2,one,two
key1,,
a,-0.864172,-0.704522
b,-2.145779,1.645362


#### 分组键是任何长度适当的数组：

In [11]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005,2005,2006,2005,2006])

In [14]:
df['data1'].groupby([states,years]).mean().unstack()

,2005,2006
California,-0.704522,-2.145779
Ohio,-0.209198,0.335414


#### 分组键是DataFrame的某个列名的值：其他列是分组信息(不是数值数据的列被排除)

In [17]:
df.groupby('key1').mean()
df.groupby(['key1','key2']).mean()

,data1,data2
key1,,
a,-0.810956,0.299147
b,-0.250209,-0.455821


data1     data2
key1 key2                    
a    one  -0.864172 -0.141347
     two  -0.704522  1.180135
b    one  -2.145779  0.138698
     two   1.645362 -1.050341

#### GroupBy的size方法：返回一个含有分组大小的Series

In [18]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

#### 产生一组二元元组（由分组名和数据块组成）

##### 单键

In [29]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)
df.groupby('key1')
list(df.groupby('key1'))
#dict(df.groupby('key1'))
dict(list(df.groupby('key1')))

a
  key1 key2     data1     data2
0    a  one -2.063758 -1.030364
1    a  two -0.704522  1.180135
4    a  one  0.335414  0.747669
b
  key1 key2     data1     data2
2    b  one -2.145779  0.138698
3    b  two  1.645362 -1.050341


[('a',   key1 key2     data1     data2
  0    a  one -2.063758 -1.030364
  1    a  two -0.704522  1.180135
  4    a  one  0.335414  0.747669), ('b',   key1 key2     data1     data2
  2    b  one -2.145779  0.138698
  3    b  two  1.645362 -1.050341)]

{'a':   key1 key2     data1     data2
 0    a  one -2.063758 -1.030364
 1    a  two -0.704522  1.180135
 4    a  one  0.335414  0.747669, 'b':   key1 key2     data1     data2
 2    b  one -2.145779  0.138698
 3    b  two  1.645362 -1.050341}

##### 多重键：元组的第一个元素是由键值组成的元组

In [31]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)
list(df.groupby(['key1','key2']))
dict(list(df.groupby(['key1','key2'])))

('a', 'one')
  key1 key2     data1     data2
0    a  one -2.063758 -1.030364
4    a  one  0.335414  0.747669
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.704522  1.180135
('b', 'one')
  key1 key2     data1     data2
2    b  one -2.145779  0.138698
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.645362 -1.050341


[(('a', 'one'),   key1 key2     data1     data2
  0    a  one -2.063758 -1.030364
  4    a  one  0.335414  0.747669),
 (('a', 'two'),   key1 key2     data1     data2
  1    a  two -0.704522  1.180135),
 (('b', 'one'),   key1 key2     data1     data2
  2    b  one -2.145779  0.138698),
 (('b', 'two'),   key1 key2     data1     data2
  3    b  two  1.645362 -1.050341)]

{('a', 'one'):   key1 key2     data1     data2
 0    a  one -2.063758 -1.030364
 4    a  one  0.335414  0.747669, ('a', 'two'):   key1 key2     data1     data2
 1    a  two -0.704522  1.180135, ('b', 'one'):   key1 key2     data1     data2
 2    b  one -2.145779  0.138698, ('b', 'two'):   key1 key2     data1     data2
 3    b  two  1.645362 -1.050341}

In [32]:
pieces = dict(list(df.groupby(['key1','key2'])))
pieces[('a','one')]

,key1,key2,data1,data2
0,a,one,-2.063758,-1.030364
4,a,one,0.335414,0.747669


#### dtype 对列进行分组

In [35]:
df.dtypes
for dtpye,group in df.groupby(df.dtypes,axis=1):
    print(dtpye)
    print(group)

key1      object
key2      object
data1    float64
data2    float64
dtype: object

float64
      data1     data2
0 -2.063758 -1.030364
1 -0.704522  1.180135
2 -2.145779  0.138698
3  1.645362 -1.050341
4  0.335414  0.747669
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或列的子集

In [41]:
type(df[['data2']])
type(df['data2'])

pandas.core.frame.DataFrame

pandas.core.series.Series

#### 传入列表或数组，返回一个已分组的DataFrame

In [42]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.141347
     two   1.180135
b    one   0.138698
     two  -1.050341

#### 传入标量形式的单个列名，返回已分组的Series

In [45]:
df.groupby(['key1','key2'])['data2'].mean()

key1  key2
a     one    -0.141347
      two     1.180135
b     one     0.138698
      two    -1.050341
Name: data2, dtype: float64

### 通过字典或 Series 进行分组

In [53]:
people = pd.DataFrame(np.random.randn(5,5),columns=['a','b','c','d','e'],index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2:3,[1,2]]=np.nan
people
#已知列的分组关系
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

,a,b,c,d,e
Joe,1.136380,-1.114677,-0.889857,-1.127341,-2.035308
Steve,-0.514623,0.216382,-2.094553,1.539698,0.275906
Wes,-0.575157,NaN,NaN,-0.130794,-1.494209
Jim,-0.237488,-0.150140,-0.728747,-0.316915,-0.899779
Travis,-1.181142,0.723517,0.850592,1.320182,0.917310


#### 通过字典进行分组

In [54]:
by_column = people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,-2.017198,-2.013604
Steve,-0.554854,-0.022335
Wes,-0.130794,-2.069366
Jim,-1.045661,-1.287407
Travis,2.170774,0.459685


#### 通过Series进行分组

In [55]:
map_series = pd.Series(mapping)
people.groupby(map_series,axis=1).sum()

,blue,red
Joe,-2.017198,-2.013604
Steve,-0.554854,-0.022335
Wes,-0.130794,-2.069366
Jim,-1.045661,-1.287407
Travis,2.170774,0.459685


### 通过函数进行分组

In [56]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.323735,-1.264817,-1.618604,-1.575050,-4.429296
5,-0.514623,0.216382,-2.094553,1.539698,0.275906
6,-1.181142,0.723517,0.850592,1.320182,0.917310


#### 函数和列表混合分组

In [57]:
key_list = ['one','one','one','two','two']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -0.575157 -1.114677 -0.889857 -1.127341 -2.035308
  two -0.237488 -0.150140 -0.728747 -0.316915 -0.899779
5 one -0.514623  0.216382 -2.094553  1.539698  0.275906
6 two -1.181142  0.723517  0.850592  1.320182  0.917310

### 根据索引级别分组

In [58]:
#建立多级别的列
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df
                                     

cty          US                            JP          
tenor         1         3         5         1         3
0      0.915530  0.969194  0.726777 -1.801672  1.411649
1     -1.270654 -0.447816 -0.088516  0.274237  1.384671
2      1.091082  1.433674  1.246258  1.239588  1.213401
3     -1.651973  0.048488  1.627709  0.537992  2.518675

In [59]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2数据集合

#### 自己发明的聚合函数

In [62]:
df.groupby('key1')['data1'].quantite(0.9)

AttributeError: 'SeriesGroupBy' object has no attribute 'quantite'

In [84]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
df.groupby('key1')['data1'].agg(peak_to_peak)

key1
a    2.399172
b    3.791141
Name: data1, dtype: float64

In [70]:
df.groupby('key1')['data2'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.299147,1.171518,-1.030364,-0.141347,0.747669,0.963902,1.180135
b,2.0,-0.455821,0.840778,-1.050341,-0.753081,-0.455821,-0.158562,0.138698


### 面向列的多函数应用

In [78]:
tips = pd.read_csv(r"F:\DUFE\2ND semester\python\pythonforDA\examples\tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [80]:
#加一个小费百分比的列
tips['tip_pct']=tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [88]:
#根据day和smoker两列作为分组键进行聚合
grouped = tips.groupby(['day','smoker'])
#选tip_pct列作为分组信息的列
grouped_pct = grouped['tip_pct']
grouped_pct
#进行聚合
#以字符串的形式传入一个函数
grouped_pct.agg('mean')
#以列表的形式传入一组函数,自定义的不加引号
grouped_pct.agg(['mean','std',peak_to_peak])

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### 自定义聚合函数的名字

In [89]:
grouped_pct.agg([('pingjunzhi','mean'),('biaozhuncha','std')])

pingjunzhi  biaozhuncha
day  smoker                         
Fri  No        0.151650     0.028123
     Yes       0.174783     0.051293
Sat  No        0.158048     0.039767
     Yes       0.147906     0.061375
Sun  No        0.160113     0.042347
     Yes       0.187250     0.154134
Thur No        0.160298     0.038774
     Yes       0.163863     0.039389

#### 不同列

##### 用同一组函数：不改名

In [95]:
functions = ['count','mean','max']
result = groupbed['tip_pct','total_bill'].agg(functions)
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

##### 用同一组函数：改名

In [96]:
ftuple = [('pingjunshu','mean'),('fangcha',np.var)]
grouped[['tip_pct','total_bill']].agg(ftuple)

tip_pct           total_bill            
            pingjunshu   fangcha pingjunshu     fangcha
day  smoker                                            
Fri  No       0.151650  0.000791  18.420000   25.596333
     Yes      0.174783  0.002631  16.813333   82.562438
Sat  No       0.158048  0.001581  19.661778   79.908965
     Yes      0.147906  0.003767  21.276667  101.387535
Sun  No       0.160113  0.001793  20.506667   66.099980
     Yes      0.187250  0.023757  24.120000  109.046044
Thur No       0.160298  0.001503  17.113111   59.625081
     Yes      0.163863  0.001551  19.190588   69.808518

##### 用不同函数

In [100]:
grouped.agg({'tip':np.max,'size':'sum'})
grouped.agg({'tip_pct':[('zuixiaozhi','min'),('zuidazhi','max'),'mean','std'],'size':'sum'})
grouped.agg({'tip_pct':ftuple,'size':functions})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

tip_pct                               size
            zuixiaozhi  zuidazhi      mean       std  sum
day  smoker                                              
Fri  No       0.120385  0.187735  0.151650  0.028123    9
     Yes      0.103555  0.263480  0.174783  0.051293   31
Sat  No       0.056797  0.291990  0.158048  0.039767  115
     Yes      0.035638  0.325733  0.147906  0.061375  104
Sun  No       0.059447  0.252672  0.160113  0.042347  167
     Yes      0.065660  0.710345  0.187250  0.154134   49
Thur No       0.072961  0.266312  0.160298  0.038774  112
     Yes      0.090014  0.241255  0.163863  0.039389   40

tip_pct            size              
            pingjunshu   fangcha count      mean max
day  smoker                                         
Fri  No       0.151650  0.000791     4  2.250000   3
     Yes      0.174783  0.002631    15  2.066667   4
Sat  No       0.158048  0.001581    45  2.555556   4
     Yes      0.147906  0.003767    42  2.476190   5
Sun  No       0.160113  0.001793    57  2.929825   6
     Yes      0.187250  0.023757    19  2.578947   5
Thur No       0.160298  0.001503    45  2.488889   6
     Yes      0.163863  0.001551    17  2.352941   4

### 以“没有行索引”的形式返回聚合数据

In [102]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

## 10.3 apply：一般性的“拆分－应用－ 合并”

In [105]:
#定义一个根据tip_pct选出前5高的值的函数
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,n=5)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [110]:
#对smoker分组并用该函数调用apply：
tips.groupby('smoker').apply(top)
#无敌强大：如果传给 apply 的函数能够接受其他参数或关键字，则可以将这些内容放在函 数名后面一并传入
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [112]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill        tip       size    tip_pct
day  smoker                                                   
Fri  No     count    4.000000   4.000000   4.000000   4.000000
            mean    18.420000   2.812500   2.250000   0.151650
            std      5.059282   0.898494   0.500000   0.028123
            min     12.460000   1.500000   2.000000   0.120385
            25%     15.100000   2.625000   2.000000   0.137239
            50%     19.235000   3.125000   2.000000   0.149241
            75%     22.555000   3.312500   2.250000   0.163652
            max     22.750000   3.500000   3.000000   0.187735
     Yes    count   15.000000  15.000000  15.000000  15.000000
            mean    16.813333   2.714000   2.066667   0.174783
            std      9.086388   1.077668   0.593617   0.051293
            min      5.750000   1.000000   1.000000   0.103555
            25%     11.690000   1.960000   2.000000   0.133739
            50%     13.420000   2.500000   2.000000   0.173913
            75%     18.665000   3.240000   2.000000   0.209240
            max     40.170000   4.730000   4.000000   0.263480
Sat  No     count   45.000000  45.000000  45.000000  45.000000
            mean    19.661778   3.102889   2.555556   0.158048
            std      8.939181   1.642088   0.784960   0.039767
            min      7.250000   1.000000   1.000000   0.056797
            25%     14.730000   2.010000   2.000000   0.136240
            50%     17.820000   2.750000   2.000000   0.150152
            75%     20.650000   3.390000   3.000000   0.183915
            max     48.330000   9.000000   4.000000   0.291990
     Yes    count   42.000000  42.000000  42.000000  42.000000
            mean    21.276667   2.875476   2.476190   0.147906
            std     10.069138   1.630580   0.862161   0.061375
            min      3.070000   1.000000   1.000000   0.035638
            25%     13.405000   2.000000   2.000000   0.091797
            50%     20.390000   2.690000   2.000000   0.153624
...                       ...        ...        ...        ...
Sun  No     std      8.130189   1.224785   1.032674   0.042347
            min      8.770000   1.010000   2.000000   0.059447
            25%     14.780000   2.000000   2.000000   0.139780
            50%     18.430000   3.020000   3.000000   0.161665
            75%     25.000000   3.920000   4.000000   0.185185
            max     48.170000   6.000000   6.000000   0.252672
     Yes    count   19.000000  19.000000  19.000000  19.000000
            mean    24.120000   3.516842   2.578947   0.187250
            std     10.442511   1.261151   0.901591   0.154134
            min      7.250000   1.500000   2.000000   0.065660
            25%     17.165000   3.000000   2.000000   0.097723
            50%     23.100000   3.500000   2.000000   0.138122
            75%     32.375000   4.000000   3.000000   0.215325
            max     45.350000   6.500000   5.000000   0.710345
Thur No     count   45.000000  45.000000  45.000000  45.000000
            mean    17.113111   2.673778   2.488889   0.160298
            std      7.721728   1.282964   1.179796   0.038774
            min      7.510000   1.250000   1.000000   0.072961
            25%     11.690000   1.800000   2.000000   0.137741
            50%     15.950000   2.180000   2.000000   0.153492
            75%     20.270000   3.000000   2.000000   0.184843
            max     41.190000   6.700000   6.000000   0.266312
     Yes    count   17.000000  17.000000  17.000000  17.000000
            mean    19.190588   3.030000   2.352941   0.163863
            std      8.355149   1.113491   0.701888   0.039389
            min     10.340000   2.000000   2.000000   0.090014
            25%     13.510000   2.000000   2.000000   0.148038
            50%     16.470000   2.560000   2.000000   0.153846
            75%     19.810000   4.000000   2.000000   0.194837
            max     43.110000   5.000000   4.000000   0.241255

[64 rows x 4 columns]

### 禁止分组键

In [113]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析
cut和qcut与groupby结合使用轻松实现数据集的桶和分位数分析。

In [116]:
frame = pd.DataFrame({'data1':np.random.randn(1000),'data2':np.random.randn(1000)})
#定义要apply的函数
def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
#cut与groupby结合
quantiles = pd.cut(frame.data1,4)
##cut返回的Categorical对象可以直接传递到groupby。
grouped = frame.data2.groupby(quantiles)
#应用函数
grouped.apply(get_stats).unstack()

0     (-1.189, 0.369]
1      (0.369, 1.927]
2     (-1.189, 0.369]
3      (0.369, 1.927]
4      (0.369, 1.927]
5     (-1.189, 0.369]
6     (-1.189, 0.369]
7    (-2.753, -1.189]
8     (-1.189, 0.369]
9      (0.369, 1.927]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.753, -1.189] < (-1.189, 0.369] < (0.369, 1.927] < (1.927, 3.484]]

,count,max,mean,min
data1,,,,
"(-2.753, -1.189]",128.0,2.281150,0.009324,-2.525859
"(-1.189, 0.369]",524.0,3.060779,-0.005370,-2.812106
"(0.369, 1.927]",321.0,2.636592,-0.017264,-2.414117
"(1.927, 3.484]",27.0,2.045220,-0.231324,-2.607375


In [119]:
#qcut和group结合
grouping = pd.qcut(frame.data1,10,labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.281150,0.102892,-2.525859
1,100.0,1.908065,-0.199505,-2.812106
2,100.0,2.265015,-0.102494,-2.174716
3,100.0,2.519424,-0.046654,-2.312638
4,100.0,2.572639,0.131207,-1.907190
5,100.0,3.060779,0.000659,-2.635181
6,100.0,2.637756,0.064721,-2.011428
7,100.0,2.333722,-0.075403,-2.372227
8,100.0,2.529292,0.055977,-2.414117


### 示例：用特定于分组的值填充缺失值:fillna
用平均值填充NA

In [120]:
s = pd.Series(np.random.randn(6))
s[::2]=np.nan
s
s.fillna(s.mean())

0         NaN
1   -0.992602
2         NaN
3    0.839013
4         NaN
5   -0.815150
dtype: float64

0   -0.322913
1   -0.992602
2   -0.322913
3    0.839013
4   -0.322913
5   -0.815150
dtype: float64

#### 对不同的分组填充不同的值
一种方法是将数据分组，并使用 apply 和一个能够对各数据块调用 fillna 的函数即可

In [125]:
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East']*4+['West']*4
data = pd.Series(np.random.randn(8),index=states)
data
data[['Vermont', 'Nevada', 'Idaho']]=np.nan
data


Ohio          0.623502
New York     -0.521573
Vermont      -0.958597
Florida      -0.338390
Oregon       -0.844023
Nevada        1.437214
California    0.696009
Idaho         0.826333
dtype: float64

Ohio          0.623502
New York     -0.521573
Vermont            NaN
Florida      -0.338390
Oregon       -0.844023
Nevada             NaN
California    0.696009
Idaho              NaN
dtype: float64

East   -0.078821
West   -0.074007
dtype: float64

In [127]:
#分组平均
data.groupby(group_key).mean()
#分组平均填充NA值
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

East   -0.078821
West   -0.074007
dtype: float64

Ohio          0.623502
New York     -0.521573
Vermont      -0.078821
Florida      -0.338390
Oregon       -0.844023
Nevada       -0.074007
California    0.696009
Idaho        -0.074007
dtype: float64

In [134]:
#在代码中预定义各组的填充值
fill_values = {'East':0.5,'West':-1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

AttributeError: 'list' object has no attribute 'name'

### 示例：随机采样和排列

In [139]:
suits = ['H','S','C','D']
card_val = (list(range(1,11))+[10]*3)*4
base_names = ['A']+list(range(2,11))+['J','K','Q']

In [138]:
cards=[]
for suit in ['H','S','C','D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val,index=cards)
def draw(deck, n=5): 
    return deck.sample(n)
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

### 示例：分组加权平均数和相关系数

In [143]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'],'data': np.random.randn(8), 'weights': np.random.rand(8)})
#利用 category 计算分组加权平均数：
grouped = df.groupby('category')
grouped.mean()
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

,data,weights
category,,
a,0.445733,0.442640
b,-0.469801,0.581692


category
a    1.062048
b   -0.377482
dtype: float64

#### 相关系数

In [145]:
close_px = pd.read_csv(r"F:\DUFE\2ND semester\python\pythonforDA\examples\stock_px_2.csv", parse_dates=True,index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


### 示例：组级别的线性回归

## 10.4透视表和交叉表

In [146]:
#根据 day 和 smoker 计算分组平均数,（pivot_table 的默认聚合类型），并将 day 和 smoker 放到行上：
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [147]:
#我们只想聚合 tip_pct 和size，而且想根据 time 进行分组。我将 smoker 放到列上，把 day 放到行上：
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [148]:
#传入 margins=True 添加分项小计
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [149]:
#使用 count 或 len 可 以得到有关分组大小的交叉表（计数或频率）
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

#### 交叉表